In [72]:
import numpy as np
import photoncount
import matplotlib.pyplot as pl
from ipywidgets import VBox, FloatSlider, FloatRangeSlider, HBox, BoundedFloatText, RadioButtons
import bqplot.pyplot as plt

In [201]:
ph = photoncount.Photocount()

properties_dict = {}
properties_dict['D'] = 4.0
properties_dict['lmin'] = 854.0
properties_dict['lmax'] = 855.0
properties_dict['polarimetry'] = 0
properties_dict['R'] = 8e4
properties_dict['T'] = 0.1
properties_dict['SN'] = 1e3
properties_dict['v'] = 7.0
properties_dict['binning'] = 1.0

ph.set_properties(properties_dict)
ph.compute()

In [202]:
fig1 = plt.figure(title='Spectrum', legend_location='bottom-right')
fig1.layout.width = '600px'
fig1.layout.height = '400px'
line1a = plt.plot(ph.ll * ph.m_to_nm, ph.Ilambda, 'b', stroke_width=3, figure=fig1, labels=["Atlas"])
line1b = plt.plot(ph.ll * ph.m_to_nm, ph.Ilambdac, 'r', stroke_width=3, figure=fig1, labels=["Degraded"])
plt.legend()

fig2 = plt.figure(title='Optimal pixel size')
fig2.layout.width = '600px'
fig2.layout.height = '400px'
line2 = plt.plot(ph.ll * ph.m_to_nm, ph.dx, 'b', stroke_width=3, figure=fig2)

fig3 = plt.figure(title='Integration time')
fig3.layout.width = '600px'
fig3.layout.height = '400px'
line3a = plt.plot(ph.ll * ph.m_to_nm, ph.t, 'b', stroke_width=3, figure=fig3)
line3b = plt.plot(ph.ll * ph.m_to_nm, ph.tideal, 'r', stroke_width=3, figure=fig3)

fig4 = plt.figure(title='Optimal integration time')
fig4.layout.width = '600px'
fig4.layout.height = '400px'
line4 = plt.plot(ph.ll * ph.m_to_nm, ph.dt, 'b', stroke_width=3, figure=fig4)

top = HBox([fig1, fig2])
bottom = HBox([fig3, fig4])
figs = VBox([top, bottom])

In [203]:
wid_D = BoundedFloatText(
    value=4.0,
    min=0.01,
    max=8.0,
    step=0.1,
    description='D [m]')

wid_lmin = BoundedFloatText(value=854,
    min=390,
    max=1000.0,    
    description='lmin')

wid_lmax = BoundedFloatText(value=855,
    min=390,
    max=1000.0,
    description='lmax')

wid_l = HBox([wid_lmin, wid_lmax])

wid_lmin_slider = FloatSlider(value=854,min=390,max=1000.0, step=0.05,description='lmin_slider')
wid_lmax_slider = FloatSlider(value=855,min=390,max=1000.0, step=0.05,description='lmax_slider')

wid_l_slider = HBox([wid_lmin_slider, wid_lmax_slider])

wid_pol = RadioButtons(options=['No', 'Yes'], description='Polarization')
wid_bin = RadioButtons(options=['1x1', '2x2', '4x4'], description='Binning')

wid_R = BoundedFloatText(
    value=80000.0,
    min=0.0,
    max=1e6,
    step=0.1,
    description='Spectral resolution')

wid_T = BoundedFloatText(
    value=0.1,
    min=0.0,    
    max=1.0,
    step=0.01,
    description='Total transmission')

wid_SN = BoundedFloatText(
    value=1000.0,
    min=0.0,
    max=100000,
    description='Desired S/N')

wid_v = BoundedFloatText(
    value=7.0,
    min=0.0,
    max=200,
    description='Evolution speed [km/s]')

widgets = VBox([wid_D, wid_l, wid_pol, wid_bin, wid_R, wid_T, wid_SN, wid_v])
#widgets = VBox([wid_D, wid_l, wid_l_slider, wid_pol, wid_bin, wid_R, wid_T, wid_SN, wid_v])

In [204]:
def update(change):    
    if (change.owner.description == 'lmin_slider'):
        if (wid_lmin.value > wid_lmax.value):
            wid_lmax.value = wid_lmin.value + 1.0
            
        wid_lmin.value = wid_lmin_slider.value
        if (wid_lmax.value - wid_lmin.value > 5.0):
            wid_lmax.value = wid_lmin.value + 5
            wid_lmax_slider.value = wid_lmax.value
            
    if (change.owner.description == 'lmax_slider'):
        if (wid_lmax.value < wid_lmin.value):
            wid_lmin.value = wid_lmax.value - 1.0
            
        wid_lmax.value = wid_lmax_slider.value
        if (wid_lmax.value - wid_lmin.value > 5.0):
            wid_lmin.value = wid_lmax.value - 5
            wid_lmin_slider.value = wid_lmin.value
                    
    properties_dict['D'] = wid_D.value    
    properties_dict['lmin'] = wid_lmin.value
    properties_dict['lmax'] = wid_lmax.value
    properties_dict['R'] = wid_R.value
    properties_dict['T'] = wid_T.value
    properties_dict['SN'] = wid_SN.value
    properties_dict['v'] = wid_v.value         
        
    if (wid_pol.value == 'Yes'):
        properties_dict['polarimetry'] = 1.0
    else:
        properties_dict['polarimetry'] = 0.0
    
    if (wid_bin.value == '1x1'):
        properties_dict['binning'] = 1.0
        
    if (wid_bin.value == '2x2'):
        properties_dict['binning'] = 2.0
        
    if (wid_bin.value == '4x4'):
        properties_dict['binning'] = 4.0
                    
    ph.set_properties(properties_dict)
    ph.compute()
    
    line1a.x = ph.ll * ph.m_to_nm
    line1a.y = ph.Ilambda
    line1b.x = ph.ll * ph.m_to_nm
    line1b.y = ph.Ilambdac
    
    line2.x = ph.ll * ph.m_to_nm
    line2.y = ph.dx
    
    line3a.x = ph.ll * ph.m_to_nm
    line3a.y = ph.t
    line3b.x = ph.ll * ph.m_to_nm
    line3b.y = ph.tideal
    
    line4.x = ph.ll * ph.m_to_nm
    line4.y = ph.dt
    
# register the above callback with the 'value' trait of the sliders
for slider in [wid_D, wid_lmin, wid_lmax, wid_lmin_slider, wid_lmax_slider, wid_R, wid_T, wid_SN, wid_v, wid_pol, wid_bin]:
    slider.observe(update, 'value')

In [205]:
VBox([widgets, figs])